## Basic Flow

This notebook describes a basic flow of ml-git with api. There you will learn how to initialize an ml-git project, how to perform all necessary configuration steps and how to version a dataset. We will divide this quick howto into 3 main sections:

##### ml-git repository configuation/intialization 
     This section explains how to initialize and configure a repository for ml-git. 
##### versioning a dataset
    Having a repository initialized, this section explains how to create and upload a dataset to the store. 
##### downloading a dataset
    This section describes how to download a versioned data set using ml-git.

### 1 - ml-git repository configuation/intialization 

To start using the ml-git api we need to import it into our script

In [ ]:
from ml_git import api

To use ml-git, it is necessary to configure storages and remotes that will be used in a project. This configuration can be done through a sequence of commands, or if you already have a git repository with the stored settings, you can run the clone command to import those settings.

In this notebook we will consider the scenario of a user who wants to configure their project from scratch. The first step is to define that the directory we are working on will be an ml-git project, for this we execute the following command:

In [ ]:
api.init('repository')

After initializing an ml-git project, it is necessary that you inform the remotes and storages that will be used by the entities to be versioned. If you want to better understand why ml-git uses these resources, please take a look at the [architecture and internals documentation](../../mlgit_internals.md).

In this notebook we will configure our ml-git project with a local git repository and a local minio as storage. For this, the following commands are necessary:

In [ ]:
remote_url = '/local_server.git/'
bucket_name= 'mlgit'
end_point = 'http://127.0.0.1:9000'

# The type of entity we are working on
entity_type = 'dataset'

api.remote_add(entity_type, remote_url)
api.store_add(bucket_name, endpoint_url=end_point)

Last but not least, initialize the metadata repository

In [ ]:
api.init(entity_type)

### 2 - versioning a dataset

After the entities have been initialized and are ready for use. We can continue with the process to version our first dataset.

ml-git expects any dataset to be specified under *dataset/* directory of your project and it expects a specification file with the name of the dataset.

To create this specification file for a new entity you must run the following command:

In [ ]:
# The entity name we are working on
entity_name = 'dataset-ex'

api.create(entity_type, entity_name, categories=['computer-vision', 'images'], mutability='strict', bucket_name=bucket_name)

Once we create our dataset entity we can add the data to be versioned within the entity's directory. For this, the following code generate a new file in our dataset path.

In [ ]:
import os

def create_file(file_name='file'):
    file_path = os.path.join('dataset', 'dataset-ex', 'data', file_name)
    open(file_path, 'a').close()

create_file()

We can now proceed with the necessary steps to send the new data to storage.

In [ ]:
api.add(entity_type, entity_name, bumpversion=True)

After add the files, you need commit the metadata to the local repository. For this purpose type the following command:

In [ ]:
# Custom commit message
message = 'Commit example'

api.commit(entity_type, entity_name, message)

Last but not least, ml-git dataset push will update the remote metadata repository just after storing all actual data under management in the specified remote data store.

In [ ]:
api.push(entity_type, entity_name)

As you can observe, ml-git follows very similar workflows as for git.

### 3 - downloading a dataset

Once you have an entity versioned by ml-git, and being within an initialized directory, it is really simple to obtain data from a specific entity. As an example, in this notebook we will checkout an entity that was previously versioned, the mnist. For this, the following command is necessary:

In [ ]:
entity_name = 'mnist'

data_path = api.checkout(entity_type, entity_name, version=1)

Getting the data will auto-create a directory structure under dataset directory. That structure *computer-vision/images* is actually coming from the categories defined in the dataset spec file. Doing that way allows for easy download of many datasets in one single ml-git project without creating any conflicts.

Now the user can perform the processes he wants with the data that was downloaded in the workspace.